## 相关性分析

### 1. 处理测试集

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

# 全局变量
cols = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # 选取属性
feanum=len(cols) - 3 # 特征数量
window=5 # 时间窗设置

test_df = pd.read_csv("../data/biClsData/testData.csv", usecols=cols)
# test_df = pd.read_csv("../data/testData.csv")
test_stocks_code = test_df["kdcode"].unique()
test_stocks_num = len(test_stocks_code)

"测试集股票数量：",test_stocks_num

/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Duplicate key in file PosixPath('/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc'), line 400 ('axes.unicode_minus: True  # use Unicode for the minus symbol rather than hyphen.  See')
/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


('测试集股票数量：', 3790)

In [2]:
# 根据股票代码划分数据
for i, stock_i in enumerate(tqdm(test_stocks_code)):
    stock_i_data = test_df[test_df['kdcode'].isin([stock_i])]
    exec("test_df%s = stock_i_data" % i)

100%|██████████| 3790/3790 [00:56<00:00, 66.66it/s]


In [3]:
testResult = []
for i in tqdm(range(test_stocks_num)): # 遍历训练集所有股票的DataFrame
    exec("testData = test_df%s.values" % i)
    sequence_length = window + 1
    for index in range(len(testData) - sequence_length + 1):
        testResult.append(testData[index: index + sequence_length])

testResult = np.array(testResult)
testResult.shape

100%|██████████| 3790/3790 [00:01<00:00, 2227.78it/s]


(1280967, 6, 21)

In [4]:
X_test = testResult[:, :-1, 2:-1]
X_test = X_test.astype('float64')
print("测试集X：" + str(X_test.shape))

测试集X：(1280967, 5, 18)


In [5]:
Y_test = testResult[:, window, -1]
Y_test.shape

(1280967,)

In [6]:
Y_test = Y_test.astype('float64')

### 2. 载入模型

In [7]:
from keras.models import load_model

model_path = './models/biClsModel_D1.h5'
model = load_model(model_path)

In [8]:
#在测试集上的预测
Y_test_predict=model.predict(X_test)[:,0]
Y_test=Y_test

In [9]:
Y_test_predict

array([0.5039813 , 0.5005958 , 0.48422447, ..., 0.50467294, 0.538943  ,
       0.5045809 ], dtype=float32)

In [10]:
# pred_df = pd.DataFrame(testResult[:, -1, :2], columns = ['dt', 'kdcode'])
# pred_df
pred_df = pd.DataFrame(testResult[:, -1, :], columns = test_df.columns)
pred_df

,dt,kdcode,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,vwap_mean20,...,chg_mean3,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,high_low,close_open,label
0,2020-01-09,000004.SZ,0.0175437,0.0365156,0.0175682,0.250903,0.249627,0.248677,0.251511,0.242408,...,0.0444631,0.0429654,0.0435066,0.449289,0.445876,0.448441,0.454036,0.0739537,0.594042,1
1,2020-01-10,000004.SZ,0.0243435,0.0510284,0.0243787,0.252735,0.251307,0.249019,0.250804,0.243857,...,0.0435665,0.04295,0.0421521,0.45728,0.450658,0.44922,0.455061,0.0686713,0.570711,1
2,2020-01-13,000004.SZ,0.0496388,0.106819,0.0497138,0.259645,0.25401,0.251788,0.251117,0.245677,...,0.046193,0.0449511,0.0433369,0.45728,0.454689,0.45045,0.45254,0.144738,0.613689,1
3,2020-01-14,000004.SZ,0.027539,0.0593994,0.0275788,0.260263,0.257132,0.254303,0.251414,0.247508,...,0.0446974,0.044353,0.043284,0.466706,0.460496,0.455205,0.454446,0.0644454,0.583605,1
4,2020-01-15,000004.SZ,0.0205505,0.0434564,0.02058,0.255019,0.257894,0.255,0.251502,0.248837,...,0.0435504,0.0441781,0.0429476,0.467935,0.464048,0.459755,0.455122,0.0707843,0.552292,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280962,2021-07-26,688600.SH,0.00559197,0.00811681,0.00689352,0.172849,0.17406,0.173798,0.177549,0.177473,...,0.0425887,0.0422379,0.0421485,0.0502832,0.050765,0.0508334,0.0520891,0.0050244,0.581775,1
1280963,2021-07-27,688600.SH,0.00970434,0.0144735,0.0120379,0.177792,0.174699,0.174523,0.176799,0.177293,...,0.043179,0.0433933,0.0419921,0.0506661,0.0507186,0.0506315,0.0518593,0.0104077,0.583235,1
1280964,2021-07-28,688600.SH,0.00709277,0.0102393,0.00876422,0.171868,0.1737,0.173852,0.175298,0.176856,...,0.0418379,0.0421967,0.0415828,0.0512229,0.0507186,0.0508055,0.051525,0.0127405,0.576352,0
1280965,2021-07-29,688600.SH,0.00616996,0.00899816,0.00759503,0.173699,0.173983,0.173415,0.174238,0.176642,...,0.0429154,0.0425165,0.0420161,0.0502136,0.0506954,0.0506872,0.0510689,0.00645995,0.581775,0


In [11]:
pred_df['score'] = Y_test_predict
pred_df

,dt,kdcode,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,vwap_mean20,...,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,high_low,close_open,label,score
0,2020-01-09,000004.SZ,0.0175437,0.0365156,0.0175682,0.250903,0.249627,0.248677,0.251511,0.242408,...,0.0429654,0.0435066,0.449289,0.445876,0.448441,0.454036,0.0739537,0.594042,1,0.503981
1,2020-01-10,000004.SZ,0.0243435,0.0510284,0.0243787,0.252735,0.251307,0.249019,0.250804,0.243857,...,0.04295,0.0421521,0.45728,0.450658,0.44922,0.455061,0.0686713,0.570711,1,0.500596
2,2020-01-13,000004.SZ,0.0496388,0.106819,0.0497138,0.259645,0.25401,0.251788,0.251117,0.245677,...,0.0449511,0.0433369,0.45728,0.454689,0.45045,0.45254,0.144738,0.613689,1,0.484224
3,2020-01-14,000004.SZ,0.027539,0.0593994,0.0275788,0.260263,0.257132,0.254303,0.251414,0.247508,...,0.044353,0.043284,0.466706,0.460496,0.455205,0.454446,0.0644454,0.583605,1,0.495291
4,2020-01-15,000004.SZ,0.0205505,0.0434564,0.02058,0.255019,0.257894,0.255,0.251502,0.248837,...,0.0441781,0.0429476,0.467935,0.464048,0.459755,0.455122,0.0707843,0.552292,0,0.478414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280962,2021-07-26,688600.SH,0.00559197,0.00811681,0.00689352,0.172849,0.17406,0.173798,0.177549,0.177473,...,0.0422379,0.0421485,0.0502832,0.050765,0.0508334,0.0520891,0.0050244,0.581775,1,0.566106
1280963,2021-07-27,688600.SH,0.00970434,0.0144735,0.0120379,0.177792,0.174699,0.174523,0.176799,0.177293,...,0.0433933,0.0419921,0.0506661,0.0507186,0.0506315,0.0518593,0.0104077,0.583235,1,0.546640
1280964,2021-07-28,688600.SH,0.00709277,0.0102393,0.00876422,0.171868,0.1737,0.173852,0.175298,0.176856,...,0.0421967,0.0415828,0.0512229,0.0507186,0.0508055,0.051525,0.0127405,0.576352,0,0.504673
1280965,2021-07-29,688600.SH,0.00616996,0.00899816,0.00759503,0.173699,0.173983,0.173415,0.174238,0.176642,...,0.0425165,0.0420161,0.0502136,0.0506954,0.0506872,0.0510689,0.00645995,0.581775,0,0.538943


In [12]:
pred_df['dt'] = pd.to_datetime(pred_df['dt'])
pred_df

,dt,kdcode,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,vwap_mean20,...,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,high_low,close_open,label,score
0,2020-01-09,000004.SZ,0.0175437,0.0365156,0.0175682,0.250903,0.249627,0.248677,0.251511,0.242408,...,0.0429654,0.0435066,0.449289,0.445876,0.448441,0.454036,0.0739537,0.594042,1,0.503981
1,2020-01-10,000004.SZ,0.0243435,0.0510284,0.0243787,0.252735,0.251307,0.249019,0.250804,0.243857,...,0.04295,0.0421521,0.45728,0.450658,0.44922,0.455061,0.0686713,0.570711,1,0.500596
2,2020-01-13,000004.SZ,0.0496388,0.106819,0.0497138,0.259645,0.25401,0.251788,0.251117,0.245677,...,0.0449511,0.0433369,0.45728,0.454689,0.45045,0.45254,0.144738,0.613689,1,0.484224
3,2020-01-14,000004.SZ,0.027539,0.0593994,0.0275788,0.260263,0.257132,0.254303,0.251414,0.247508,...,0.044353,0.043284,0.466706,0.460496,0.455205,0.454446,0.0644454,0.583605,1,0.495291
4,2020-01-15,000004.SZ,0.0205505,0.0434564,0.02058,0.255019,0.257894,0.255,0.251502,0.248837,...,0.0441781,0.0429476,0.467935,0.464048,0.459755,0.455122,0.0707843,0.552292,0,0.478414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280962,2021-07-26,688600.SH,0.00559197,0.00811681,0.00689352,0.172849,0.17406,0.173798,0.177549,0.177473,...,0.0422379,0.0421485,0.0502832,0.050765,0.0508334,0.0520891,0.0050244,0.581775,1,0.566106
1280963,2021-07-27,688600.SH,0.00970434,0.0144735,0.0120379,0.177792,0.174699,0.174523,0.176799,0.177293,...,0.0433933,0.0419921,0.0506661,0.0507186,0.0506315,0.0518593,0.0104077,0.583235,1,0.546640
1280964,2021-07-28,688600.SH,0.00709277,0.0102393,0.00876422,0.171868,0.1737,0.173852,0.175298,0.176856,...,0.0421967,0.0415828,0.0512229,0.0507186,0.0508055,0.051525,0.0127405,0.576352,0,0.504673
1280965,2021-07-29,688600.SH,0.00616996,0.00899816,0.00759503,0.173699,0.173983,0.173415,0.174238,0.176642,...,0.0425165,0.0420161,0.0502136,0.0506954,0.0506872,0.0510689,0.00645995,0.581775,0,0.538943


### 3. 计算`当天开盘价 - 前一天收盘价`

In [13]:
df = pd.read_csv('../data/data2.csv')
df['dt'] = pd.to_datetime(df['dt'])
df = df[['dt', 'kdcode', 'adj_open', 'adj_close']]
df

,dt,kdcode,adj_open,adj_close
0,2016-01-04,000004.SZ,270.239004,243.215104
1,2016-01-05,000004.SZ,220.783500,227.730821
2,2016-01-06,000004.SZ,227.259816,230.674601
3,2016-01-07,000004.SZ,226.671060,207.595366
4,2016-01-08,000004.SZ,206.829983,208.890629
...,...,...,...,...
4122506,2021-07-26,688600.SH,15.530000,15.590000
4122507,2021-07-27,688600.SH,15.640000,15.840000
4122508,2021-07-28,688600.SH,15.800000,15.340000
4122509,2021-07-29,688600.SH,15.510000,15.570000


In [14]:
df = df[df['dt'] >= '2020']
df

,dt,kdcode,adj_open,adj_close
776,2020-01-02,000004.SZ,133.353234,131.469215
777,2020-01-03,000004.SZ,131.351464,131.586966
778,2020-01-06,000004.SZ,130.703832,128.054430
779,2020-01-07,000004.SZ,128.348808,130.115076
780,2020-01-08,000004.SZ,128.996440,128.231057
...,...,...,...,...
4122506,2021-07-26,688600.SH,15.530000,15.590000
4122507,2021-07-27,688600.SH,15.640000,15.840000
4122508,2021-07-28,688600.SH,15.800000,15.340000
4122509,2021-07-29,688600.SH,15.510000,15.570000


In [15]:
res = df['adj_open'].iloc[1:].values - df['adj_close'].iloc[:-1].values
res

array([-0.1177512, -0.883134 ,  0.294378 , ..., -0.04     ,  0.17     ,
       -0.13     ])

In [16]:
res = list(res)
res.append(0)
len(res)

1299899

In [17]:
df['open_close'] = res
df

/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,dt,kdcode,adj_open,adj_close,open_close
776,2020-01-02,000004.SZ,133.353234,131.469215,-0.117751
777,2020-01-03,000004.SZ,131.351464,131.586966,-0.883134
778,2020-01-06,000004.SZ,130.703832,128.054430,0.294378
779,2020-01-07,000004.SZ,128.348808,130.115076,-1.118636
780,2020-01-08,000004.SZ,128.996440,128.231057,1.942895
...,...,...,...,...,...
4122506,2021-07-26,688600.SH,15.530000,15.590000,0.050000
4122507,2021-07-27,688600.SH,15.640000,15.840000,-0.040000
4122508,2021-07-28,688600.SH,15.800000,15.340000,0.170000
4122509,2021-07-29,688600.SH,15.510000,15.570000,-0.130000


### 4. 计算`当天开盘价 - 前一天开盘价`

In [18]:
res = df['adj_open'].iloc[1:].values - df['adj_open'].iloc[:-1].values
res

array([-2.0017704, -0.6476316, -2.355024 , ...,  0.16     , -0.29     ,
       -0.07     ])

In [19]:
res = list(res)
res.append(0)
len(res)

1299899

In [20]:
df['open_open'] = res
df

/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,dt,kdcode,adj_open,adj_close,open_close,open_open
776,2020-01-02,000004.SZ,133.353234,131.469215,-0.117751,-2.001770
777,2020-01-03,000004.SZ,131.351464,131.586966,-0.883134,-0.647632
778,2020-01-06,000004.SZ,130.703832,128.054430,0.294378,-2.355024
779,2020-01-07,000004.SZ,128.348808,130.115076,-1.118636,0.647632
780,2020-01-08,000004.SZ,128.996440,128.231057,1.942895,1.177512
...,...,...,...,...,...,...
4122506,2021-07-26,688600.SH,15.530000,15.590000,0.050000,0.110000
4122507,2021-07-27,688600.SH,15.640000,15.840000,-0.040000,0.160000
4122508,2021-07-28,688600.SH,15.800000,15.340000,0.170000,-0.290000
4122509,2021-07-29,688600.SH,15.510000,15.570000,-0.130000,-0.070000


### 5. 合并`df`和`pred_df`

In [21]:
df.set_index(['dt', 'kdcode'],inplace = True)
df

,,adj_open,adj_close,open_close,open_open
dt,kdcode,,,,
2020-01-02,000004.SZ,133.353234,131.469215,-0.117751,-2.001770
2020-01-03,000004.SZ,131.351464,131.586966,-0.883134,-0.647632
2020-01-06,000004.SZ,130.703832,128.054430,0.294378,-2.355024
2020-01-07,000004.SZ,128.348808,130.115076,-1.118636,0.647632
2020-01-08,000004.SZ,128.996440,128.231057,1.942895,1.177512
...,...,...,...,...,...
2021-07-26,688600.SH,15.530000,15.590000,0.050000,0.110000
2021-07-27,688600.SH,15.640000,15.840000,-0.040000,0.160000
2021-07-28,688600.SH,15.800000,15.340000,0.170000,-0.290000


In [22]:
pred_df.set_index(['dt', 'kdcode'], inplace = True)
pred_df

,,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,vwap_mean20,pct_chg,chg_mean3,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,high_low,close_open,label,score
dt,kdcode,,,,,,,,,,,,,,,,,,,,
2020-01-09,000004.SZ,0.0175437,0.0365156,0.0175682,0.250903,0.249627,0.248677,0.251511,0.242408,0.0663116,0.0444631,0.0429654,0.0435066,0.449289,0.445876,0.448441,0.454036,0.0739537,0.594042,1,0.503981
2020-01-10,000004.SZ,0.0243435,0.0510284,0.0243787,0.252735,0.251307,0.249019,0.250804,0.243857,0.0622097,0.0435665,0.04295,0.0421521,0.45728,0.450658,0.44922,0.455061,0.0686713,0.570711,1,0.500596
2020-01-13,000004.SZ,0.0496388,0.106819,0.0497138,0.259645,0.25401,0.251788,0.251117,0.245677,0.067416,0.046193,0.0449511,0.0433369,0.45728,0.454689,0.45045,0.45254,0.144738,0.613689,1,0.484224
2020-01-14,000004.SZ,0.027539,0.0593994,0.0275788,0.260263,0.257132,0.254303,0.251414,0.247508,0.0619147,0.0446974,0.044353,0.043284,0.466706,0.460496,0.455205,0.454446,0.0644454,0.583605,1,0.495291
2020-01-15,000004.SZ,0.0205505,0.0434564,0.02058,0.255019,0.257894,0.255,0.251502,0.248837,0.0588377,0.0435504,0.0441781,0.0429476,0.467935,0.464048,0.459755,0.455122,0.0707843,0.552292,0,0.478414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-26,688600.SH,0.00559197,0.00811681,0.00689352,0.172849,0.17406,0.173798,0.177549,0.177473,0.061494,0.0425887,0.0422379,0.0421485,0.0502832,0.050765,0.0508334,0.0520891,0.0050244,0.581775,1,0.566106
2021-07-27,688600.SH,0.00970434,0.0144735,0.0120379,0.177792,0.174699,0.174523,0.176799,0.177293,0.0648301,0.043179,0.0433933,0.0419921,0.0506661,0.0507186,0.0506315,0.0518593,0.0104077,0.583235,1,0.546640
2021-07-28,688600.SH,0.00709277,0.0102393,0.00876422,0.171868,0.1737,0.173852,0.175298,0.176856,0.0568099,0.0418379,0.0421967,0.0415828,0.0512229,0.0507186,0.0508055,0.051525,0.0127405,0.576352,0,0.504673


In [23]:
res_df = pd.concat([df, pred_df], axis = 1)

In [24]:
res_df.dropna(inplace = True)
res_df

adj_open   adj_close  open_close  open_open  \
dt         kdcode                                                     
2020-01-09 000004.SZ  130.173952  131.410339    1.059761   2.296148   
           000005.SZ   32.403106   32.811978   -0.102218   0.306654   
           000007.SZ  113.871744  113.751245    0.602496   0.481997   
           000008.SZ  100.508665  101.044711    0.804069   1.340116   
           000010.SZ   37.813581   39.752739    0.430924   2.370082   
...                          ...         ...         ...        ...   
2021-07-30 688586.SH   25.800000   27.690000   11.410000  13.300000   
           688588.SH   17.760000   17.490000   52.060000  51.790000   
           688589.SH   45.580000   43.000000  -13.510000 -16.090000   
           688596.SH   22.800000   23.420000   -7.410000  -6.790000   
           688600.SH   15.440000   15.550000    0.000000   0.000000   

                          volume    turnover        turn       vwap  \
dt         kdcode                                                     
2020-01-09 000004.SZ   0.0175437   0.0365156   0.0175682   0.250903   
           000005.SZ    0.223074   0.0667457   0.0174407  0.0300126   
           000007.SZ   0.0591743   0.0521899   0.0158425   0.102244   
           000008.SZ    0.430906    0.151929   0.0137403   0.036627   
           000010.SZ    0.103685   0.0355992   0.0164259  0.0354837   
...                          ...         ...         ...        ...   
2021-07-30 688586.SH    0.125584    0.317749    0.111886   0.306526   
           688588.SH   0.0582246    0.096615   0.0198715   0.198594   
           688589.SH    0.078631    0.323306    0.117966   0.502542   
           688596.SH   0.0698226     0.15546    0.101364   0.268887   
           688600.SH  0.00466634  0.00677244  0.00572433   0.172831   

                     vwap_mean3 vwap_mean5  ...  chg_mean5 chg_mean10  \
dt         kdcode                           ...                         
2020-01-09 000004.SZ   0.249627   0.248677  ...  0.0429654  0.0435066   
           000005.SZ  0.0293696  0.0287327  ...  0.0437228  0.0439564   
           000007.SZ   0.101903   0.101777  ...  0.0425546  0.0427483   
           000008.SZ  0.0360596  0.0355235  ...  0.0435173  0.0436605   
           000010.SZ  0.0338329  0.0326909  ...  0.0457262  0.0443396   
...                         ...        ...  ...        ...        ...   
2021-07-30 688586.SH   0.289387   0.297537  ...  0.0444628   0.043579   
           688588.SH   0.194729   0.198191  ...  0.0425474  0.0423462   
           688589.SH   0.501282   0.477953  ...  0.0500748  0.0473788   
           688596.SH   0.253749   0.255645  ...  0.0436822  0.0447774   
           688600.SH   0.172328   0.173008  ...  0.0427537  0.0421193   

                       adj_open open_mean3 open_mean5 open_mean10    high_low  \
dt         kdcode                                                               
2020-01-09 000004.SZ   0.449289   0.445876   0.448441    0.454036   0.0739537   
           000005.SZ   0.109008   0.109014   0.108417     0.10693   0.0146738   
           000007.SZ   0.392551    0.39345   0.394103    0.397325   0.0454078   
           000008.SZ   0.346042   0.346715   0.346073    0.340655   0.0336664   
           000010.SZ   0.127839   0.126848   0.125646    0.124576   0.0347968   
...                         ...        ...        ...         ...         ...   
2021-07-30 688586.SH  0.0860268  0.0852739   0.087142   0.0884937   0.0448608   
           688588.SH  0.0580445  0.0572398  0.0579767   0.0582273   0.0104077   
           688589.SH   0.154869   0.145879   0.138385    0.125967   0.0629845   
           688596.SH  0.0755857  0.0727305  0.0746585   0.0742953   0.0358886   
           688600.SH    0.04997  0.0504633  0.0504365   0.0507417  0.00287109   

                     close_open label     score  
dt         kdcode                                
2020-01-09 000004.SZ   0.594042     1  0.503981  
           000005.SZ  

### 5. 计算相关性

In [25]:
res_df = res_df.astype(float)
# res_df = res_df[['open_close', 'open_open', 'score']]
res_df.corr()

,adj_open,adj_close,open_close,open_open,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,...,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,high_low,close_open,label,score
adj_open,1.000000,0.999287,-0.042253,-0.049569,0.106146,0.417442,-0.012418,0.368127,0.367752,0.366978,...,0.021197,0.031265,1.000000,0.999350,0.998652,0.997063,0.771038,-0.027473,-0.022613,-0.257006
adj_close,0.999287,1.000000,-0.042371,-0.035323,0.107924,0.420134,-0.009586,0.368632,0.367762,0.366898,...,0.031504,0.038381,0.999287,0.998710,0.998048,0.996524,0.774492,0.010287,-0.004133,-0.256220
open_close,-0.042253,-0.042371,1.000000,0.924914,-0.002084,-0.024233,-0.010403,-0.021078,-0.020849,-0.020656,...,-0.001659,-0.002763,-0.042253,-0.041608,-0.041399,-0.040603,-0.041803,-0.002764,0.078506,0.017791
open_open,-0.049569,-0.035323,0.924914,1.000000,0.015619,0.003294,0.018919,-0.015644,-0.020406,-0.021134,...,0.102148,0.068969,-0.049569,-0.048240,-0.047680,-0.046281,-0.006461,0.377618,0.258788,0.025230
volume,0.106146,0.107924,-0.002084,0.015619,1.000000,0.637892,0.248039,-0.273573,-0.276310,-0.278679,...,0.139170,0.139134,0.106146,0.102656,0.099891,0.094745,0.210705,0.046160,0.027607,-0.253646
turnover,0.417442,0.420134,-0.024233,0.003294,0.637892,1.000000,0.438551,0.268569,0.263370,0.258779,...,0.194216,0.216012,0.417442,0.412075,0.407721,0.399309,0.565217,0.067685,0.021640,-0.453684
turn,-0.012418,-0.009586,-0.010403,0.018919,0.248039,0.438551,1.000000,0.237166,0.232462,0.228236,...,0.234554,0.280367,-0.012418,-0.016012,-0.018938,-0.024427,0.233798,0.075103,0.028457,-0.282041
vwap,0.368127,0.368632,-0.021078,-0.015644,-0.273573,0.268569,0.237166,1.000000,0.999290,0.998382,...,0.056289,0.081187,0.368127,0.367154,0.366190,0.363979,0.405454,0.010187,-0.009445,-0.211443
vwap_mean3,0.367752,0.367762,-0.020849,-0.020406,-0.276310,0.263370,0.232462,0.999290,1.000000,0.999600,...,0.040764,0.070855,0.367752,0.367475,0.366695,0.364565,0.403015,-0.002895,-0.017462,-0.204979
vwap_mean5,0.366978,0.366898,-0.020656,-0.021134,-0.278679,0.258779,0.228236,0.998382,0.999600,1.000000,...,0.024781,0.060309,0.366978,0.367107,0.366761,0.364949,0.400903,-0.005280,-0.018573,-0.199595
